In [1]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed, retries
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


install_requirements()
clear_output()
print("🚀 Setup complete. Continue to the next cell.")

🚀 Setup complete. Continue to the next cell.


In [ ]:
from dotenv import load_dotenv


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True, dotenv_path=".env")

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)

    print("Environment variables are set.")


setup_env()

OPENAI_API_KEY is set.
Environment variables are set.


In [4]:
from openai import OpenAI
import os
from pydantic import BaseModel
import traceback

openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


class ContextAdherenceResult(BaseModel):
    context_adherence_score: float
    context_adherence_reflection: str
    error: str


DEFAULT_COMPLETION_MODEL = "gpt-4o"
DEFAULT_DETECTION_MODEL = "gpt-4o"


def detect_context_adherence(prompt: str, context: str) -> ContextAdherenceResult:
    """Detects if the completion is relevant to the context

    Args:
        prompt : str : The prompt to provide the model.
        ontext : str : The context to provide the model.

    Returns:
        ContextAdherenceResult : The result of the context adherence detection.
    """
    try:
        response = openai.chat.completions.create(
            model=DEFAULT_COMPLETION_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant. Given a prompt and context, you should provide a response that is relevant to the context.",
                },
                {"role": "user", "content": f"Prompt: {prompt}\nContext: {context}"},
            ],
        )
        completion = response.choices[0].message.content

        response = openai.beta.chat.completions.parse(
            model=DEFAULT_DETECTION_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": f"""
                        Detect the 'Context Adherence' of the completion.
                        Provide a score between 0 and 1, where 0 means the completion is not relevant to the context and 1 means the completion is relevant to the context.
                        Also provide a reflection on why the completion is or is not relevant to the context.
                        Based on the provided context, check the following completion for the parameters:
                    """,
                },
                {
                    "role": "assistant",
                    "content": f"Prompt: {prompt}\nContext: {context}\nCompletion: {completion}",
                },
            ],
            response_format=ContextAdherenceResult,
        )
        return response.choices[0].message.parsed
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return ContextAdherenceResult(
            context_adherence_score=0,
            context_adherence_reflection="",
            error=str(e),
        )

In [ ]:
prompt = f"Provide a summary about the person Jimon's Profile."

context = """
Jimon is a software engineer who has been working in the tech industry for over 10 years. He has experience in developing web applications, mobile applications, and cloud services. He is passionate about learning new technologies and enjoys working on challenging projects. In his free time, he likes to travel and explore new places. He is also an avid reader and enjoys reading books on technology and science fiction.
Jimon graduated with a degree in Computer Science from a top university and has worked at several tech companies. He is known for his problem-solving skills and attention to detail. He is a team player and enjoys collaborating with others to build innovative solutions. Jimon is currently looking for new opportunities to work on exciting projects and expand his skill set.
Jimon is married and has two children. He values work-life balance and believes in giving back to the community. He volunteers at local schools and mentors students who are interested in pursuing a career in technology. Jimon is a positive and optimistic person who believes in continuous improvement and personal growth.
"""